# **Multi-Agent Reinforcement Learning**
Multi-agent reinforcement learning is the study of numerous artificial intelligence agents cohabitating in an environment, often collaborating toward some end goal. When focusing on collaboration, it derives inspiration from other social structures in the animal kingdom. It also draws heavily on game theory.

Multi-agent systems can be used to address problems in a variety of domains, including robotics, distributed control, telecommunications, and economics. The complexity of many tasks arising in these domains makes them difficult to solve with preprogrammed agent behaviors. The agents must instead discover a solution on their own, using learning. A significant part of the research on multi-agent learning concerns reinforcement learning techniques.

## What does this have to do with Artificial Intelligence?
If we’re working toward advancing artificial intelligence, it reasonable to posit that the same sort of sociability that is so fundamental to our own intelligence will be important to advancing artificial intelligence.

To illustrate the difficulty of this problem, when designing a single agent to play Space Invaders, it is relatively simple to create a reward and punishment system: the reward is associated in the increase in game points and the punishment is associated with the death of the agent’s spacecraft.

Whereas with multi-agent systems it can be difficult to clearly associate a reward with a particular action by a particular agent. For example, if you built a transport simulation, with 10 different agents driving a bus apiece, you could trial rewarding the agents collectively for reducing wait time across all passengers. It is incredibly difficult for the agents to map their own actions to a collective reduced wait time, as the agents can struggle to attribute which action lead to the reduction. This problem increases exponentially with the number of agents. When you factor in the city of London has 8000 buses, you begin to get an understanding of the complexity required.

## How single and multi agent reinforcment learning are relatable?
Learning to play multiplayer games represents many of the most profound achievements of artificial intelligence in our lifetimes. These accomplishments include learning to play Go, DOTA 2, and StarCraft 2 to superhuman levels of performance. Using reinforcement learning to control multiple agents, unsurprisingly, is referred to as multi-agent reinforcement learning. In general it’s the same as single agent reinforcement learning, where each agent is trying to learn it’s own policy to optimize its own reward. Using a central policy for all agents is possible, but multiple agents would have to communicate with a central server to compute their actions (which is problematic in most real world scenarios), so in practice decentralized multi-agent reinforcement learning is used. This can be visualized as follows:

![Visual representation](https://miro.medium.com/max/1050/1*1o1oeH3vpzsfJukLbFsekw.png)

## Libraries used for MARL
Gym is a famous library in reinforcement learning developed by OpenAI that provides a standard API for environments so that they can be easily learned with different reinforcement learning codebases, and so that for the same learning code base different environments can be easily tried. PettingZoo is a newer library that’s like a multi-agent version of Gym. It’s basic API usage looks like this:




In [ ]:
from pettingzoo.butterfly import pistonball_v4
env = pistonball_v4.env()
env.reset()
for agent in env.agent_iter():
    observation, reward, done, info = env.last()
    action = policy(observation, agent)
    env.step(action)

## Types of Multi-agent Reinforcement Learning(MARL)
* **Cooperative:** All agents working towards a common goal
* **Competitive:** Agents competing with one another to accomplish a goal
* **Some mix of the two:** Think a 5v5 basketball game, where individuals on the same team are coordinating with one another, but the two teams are competing against one another.


## Pistonball- An Implementaion of MARL
In it, each piston is an agent that can be separately controlled by a policy.The action the policy returns is the amount to raise or lower the piston (from -4 to 4 pixels). The goal is for the pistons to learn how to work together to roll the ball to the left wall as fast as possible. Each piston agent is rewarded negatively if the ball moves right, positively if the ball moves left, and receives a small amount of negative reward at every time step to incentivize moving to the left as fast as possible.A plethora of techniques exist to learn a single agent environment in reinforcement learning. These serve as the basis for algorithms in multi-agent reinforcement learning. The simplest and most popular way to do this is to have a single policy network shared between all agents, so that all agents use the same function to pick an action. Each agent can train this shared network using any single agent method. This is typically referred to as parameter sharing. That’s what we’ll be using today, with the PPO single agent method (one of the best methods for continuous control tasks like this).

In [ ]:
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3 import PPO
from pettingzoo.butterfly import pistonball_v4
import supersuit as ss

A few years back OpenAI released the “baselines” repository which included implementations of most of the major deep reinforcement learning algorithms. This repository was turned into the Stable Baselines library intended for beginners and practitioners of reinforcement learning to easily use to learn Gym environments. The CnnPolicy in it is just a deep convolutional neural network object that Stable Baselines includes which automatically resizes the input and output layers of the neural network to adapt to the observation and action space of the environment. SuperSuit is a package that provides preprocessing functions for both Gym and PettingZoo environments, as we’ll see below. Environments and wrappers are versioned to ensure comparisons are precisely reproducible in academic research.

In [ ]:
env = pistonball_v4.parallel_env(n_pistons=20, time_penalty=-0.1, continuous=True, random_drop=True, random_rotate=True, ball_mass=0.75, ball_friction=0.3, ball_elasticity=1.5, max_cycles=125)
env = ss.color_reduction_v0(env, mode=’B’)
env = ss.resize_v0(env, x_size=84, y_size=84)
env = ss.frame_stack_v1(env, 3)

By doing the above implementations, we have converted the full color images to grayscale images because in full color images are 3x more computationally expensive for the neural networks to process than grayscale images due to the 3 color channels.This is done wrapping the environment with SuperSuit (remember we imported it as ss above) implemented in second statement in above code. Then after that we shrinked the size of image again because it was hard to compute.The last major thing we want to do is slightly odd at first. Because the ball is on motion, we want to give the policy network an easy way of seeing how fast it’s moving and accelerating. The simplest way to do that is to stack the past few frames together as the channels of each observation. Stacking 3 together gives enough information to compute acceleration, but 4 is more standard. 

Next, we need to convert the environments API a tiny bit, which will cause Stable Baselines to do parameter sharing of the policy network on a multiagent environment (instead of learning a single-agent environment like normal).


In [ ]:
env = ss.pettingzoo_env_to_vec_env_v0(env)

Finally, we need to set the environment to run multiple versions of itself in parallel. Playing through the environment multiple times at once makes learning faster and is important to PPOs learning performance. SuperSuit offers many ways to do this and the one we want to use here is this:

In [ ]:
env = ss.concat_vec_envs_v0(env, 8, num_cpus=4, base_class=’stable_baselines3’)

Finally, we can get to some actual learning. This can be done pretty easily with Stable Baselines with three more lines of code:

In [ ]:
model = PPO(CnnPolicy, env, verbose=3, gamma=0.95, n_steps=256, ent_coef=0.0905168, learning_rate=0.00062211, vf_coef=0.042202, max_grad_norm=0.9, gae_lambda=0.99, n_epochs=5, clip_range=0.3, batch_size=256)
model.learn(total_timesteps=2000000)
model.save(“policy”)

### Watching our algorithm play
Once we’ve trained and save this model, we can load our policy and watch it play. First, let’s reinstantiate the environment, using the normal API this time:

In [ ]:
env = pistonball_v4.env()
env = ss.color_reduction_v0(env, mode=’B’)
env = ss.resize_v0(env, x_size=84, y_size=84)
env = ss.frame_stack_v1(env, 3)

model = PPO.load(“policy”)

env.reset()
for agent in env.agent_iter():
   obs, reward, done, info = env.last()
   act = model.predict(obs, deterministic=True)[0] if not done else None
   env.step(act)
   env.render()

The result will look like this:

![result](https://miro.medium.com/max/1050/1*cvmnY_qsacAZtdg9vmFy4w.gif)

## Applications of MARL
MARL has found incredible success in popular strategy games, has shown to be a catalyst for interesting emergent behavior, and will be key to the continued advancement of several emerging technologies, communication between different autonomous vehicles being one such example.

## Conclusion
Finally, the span between the capabilities of state-of-the-art Multi-agent reinforcement learning and human social capabilities underscores just how far we have to go as a research discipline to achieve advanced artificial intelligence.It can be considered as one of the most important research domains, with its relevance to language development, empathy, imagination, ethics and other qualities we associate with being human.